<a href="https://colab.research.google.com/github/AlexKressner/WS24_Supply_Chain_Optimierung/blob/main/MasterPlanning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Master Planning

In [45]:
! git clone https://github.com/AlexKressner/WS24_Supply_Chain_Optimierung

fatal: destination path 'WS24_Supply_Chain_Optimierung' already exists and is not an empty directory.


In [46]:
! pip install -q pyscipopt

In [47]:
import pandas as pd
from pyscipopt import Model, quicksum

## Daten laden

In [48]:
folder = "WS24_Supply_Chain_Optimierung/Daten/MasterPlanning"

In [49]:
# Kapazitäten
beschaffung_kapa = pd.read_csv(f"{folder}/Beschaffungskapazitaeten.csv", sep=";")
produktion_kapa = pd.read_csv(f"{folder}/Produktionskapazitaeten.csv", sep=";")

In [50]:
# Kosten
produktion_kosten = pd.read_csv(f"{folder}/Produktionskosten.csv", sep=";", decimal=",")
beschaffung_kosten = pd.read_csv(f"{folder}/Beschaffungskosten.csv", sep=";", decimal=",")
lager_kosten = pd.read_csv(f"{folder}/Lagerkosten.csv", sep=";", decimal=",")
transport_beschaffung_kosten = pd.read_csv(f"{folder}/Transportkosten_Beschaffung.csv", sep=";", decimal=",")
transport_distribution_kosten = pd.read_csv(f"{folder}/Transportkosten_Distribution.csv", sep=";", decimal=",")

In [51]:
# Direktbedarfskoeffizienten
koeffizienten = pd.read_csv(f"{folder}/Direktbedarfskoeffizienten.csv", sep=";", decimal=",")

In [52]:
# Nachfrage
nachfrage = pd.read_csv(f"{folder}/Nachfrage.csv", sep=";")

## Indexmengen

In [53]:
I = produktion_kapa["Werk"].unique().tolist() # Menge der Produktionsstandorte

In [54]:
J = nachfrage["Markt"].unique().tolist() #Menge der Märkte

In [55]:
L = beschaffung_kapa["Lieferant"].unique().tolist() # Menge der Lieferanten

In [56]:
P = lager_kosten["Produkt"].unique().tolist() # Menge der Produkte

In [57]:
VP = koeffizienten["Vorprodukt"].unique().tolist() # Menge der Vorprodukte

In [58]:
FP = koeffizienten["Fertigprodukt"].unique().tolist() # Menge der Fertigprodukte

In [59]:
A = {}
for vp in koeffizienten["Vorprodukt"].unique():
  A[vp] = koeffizienten[koeffizienten["Vorprodukt"]==vp]["Fertigprodukt"].tolist()

In [60]:
A

{'Flasche': ['Well'], 'Vitaminwasser': ['Well']}

In [61]:
T = beschaffung_kapa["Woche"].unique().tolist() # Menge der Planungsperioden (Wochen)

## Optimierungsmodell

In [62]:
scip = Model()

## Entscheidungsvariablen

In [63]:
# Beschaffungsmengen der jeweiligen Vorprodukte (Flasche und Vitaminwasser)
Z={}
for l in L:
  for i in I:
    for p in VP:
      for t in T:
        Z[l,i,p,t] = scip.addVar(vtype="C", name=f"{l},{i},{p},{t}")

In [64]:
# Produktionsmenge Fertigprodukte
X={}
for i in I:
  for t in T:
    for p in FP:
      X[i,p,t] = scip.addVar(vtype="C", name=f"{i},{p},{t}")

In [65]:
# Distributionsmenge Fertigprodukte
Y={}
for i in I:
  for j in J:
    for t in T:
      for p in FP:
        Y[i,j,p,t] = scip.addVar(vtype="C", name=f"{i},{j},{p},{t}")

In [66]:
# Bestand von Produkt (Vor- und Fertigprodukt)
B={}
for i in I:
  for p in P:
    for t in T:
      B[i,p,t] = scip.addVar(vtype="C", name=f"{i},{p},{t}")

In [67]:
# Nutzung von Zusatzkapazitäten
O={}
for i in I:
    for t in T:
      O[i,t] = scip.addVar(vtype="C", name=f"{i},{t}")

In [68]:
print('Anzahl Entscheidungsvariablen =', len(scip.getVars()))

Anzahl Entscheidungsvariablen = 360


## Parameter

In [69]:
# Produktionskosten
pc = produktion_kosten.set_index(["Werk","Produkt"]).to_dict("dict")["Produktionskosten"] # Kosten reguläre Produktion in €/ME
oc = produktion_kosten.set_index(["Werk"]).to_dict("dict")["Kosten_pro_Zusatzkapa"] # Kosten Zusatzkapazitäten in €/Kapazitätseinheit

In [70]:
# Beschaffungskosten
beschaffung_kosten.set_index(["Lieferant","Produkt"], inplace=True)
bc = beschaffung_kosten.to_dict("dict")["Beschaffungskosten"] # Kosten der Beschaffung eines Produktes in €/ME

In [71]:
# Transportkosten
transport_beschaffung_kosten.set_index(["Lieferant","Werk","Produkt"], inplace=True)
btc = transport_beschaffung_kosten.to_dict("dict")["Transportkosten"] # Kosten Transport bei Beschaffung in €/ME
transport_distribution_kosten.set_index(["Werk","Markt","Produkt"], inplace=True)
dtc = transport_distribution_kosten.to_dict("dict")["Transportkosten"] # Kosten Transport bei Distribution in €/ME

In [72]:
# Lagerkosten
lager_kosten.set_index(["Produkt"], inplace=True)
lc = lager_kosten.to_dict("dict")["Lagerkosten"] # Kosten Lagerung in €/Woche/ME

In [73]:
# Nachfrage
nachfrage.set_index(["Markt","Produkt","Woche"], inplace=True)
d = nachfrage.to_dict("dict")["Nachfragemenge"]

In [74]:
# Direktbedarfskoeffiziente
koeffizienten.set_index(["Vorprodukt","Fertigprodukt"], inplace=True)
a = koeffizienten.to_dict("dict")["Direktbedarfskoeffizient"]

In [75]:
# Kapazitäten
produktion_kapa.set_index(["Werk","Woche"], inplace=True)
pcap = produktion_kapa.to_dict("dict")["Kapaztiät"]
pcap_plus = produktion_kapa.to_dict("dict")["Max_Kapa_durch_Zusatzschichten"]
beschaffung_kapa.set_index(["Lieferant","Produkt","Woche"], inplace=True)
bcap = beschaffung_kapa.to_dict("dict")["Kapazität"]

## Zielfunktion

### Zielfunktion
Min $K = \sum_{l,i,p \in VP,t} bc_{lp} * Z_{lipt} + \sum_{l,i,p \in VP,t} tbc_{lip} * Z_{lipt} + \sum_{i,p \in FP,t} pc_{ip} * X_{ipt} + \sum_{i,p,t} lc_{p} * B_{ipt} + \sum_{i,t} oc^+_{it} * O_{it} + \sum_{i,j,p,t} dtc_{ijp} * Y_{ijpt}$

In [76]:
bc.get(('ErpoPlast', 'Vitaminwasser'),0)

0

In [77]:
# Minimierung der gesamten Kosten
scip.setObjective(
    sum(bc.get((l,p),0) * Z[l,i,p,t] for l in L for i in I for p in VP for t in T) + # Kosten Beschaffung
    sum(btc.get((l,i,p),0) * Z[l,i,p,t] for l in L for i in I for p in VP for t in T) + # Transportkosten Beschaffung
    sum(pc[i,p]*X[i,p,t] for i in I for p in FP for t in T) + # Kosten der Produktion mit regulärer Kapazität
    sum(oc[i]*O[i,t] for i in I for t in T) + # Kosten der Produktion mit zusätzlicher Kapazität
    sum(lc[p]*B[i,p,t] for i in I for p in P for t in T) + # Kosten der Lagerung
    sum(dtc[i,j,p]*Y[i,j,p,t] for i in I for j in J for p in FP for t in T), # Transportkosten Distribution
    sense="minimize"
    )

## Nebenbedingungen

**(1) Lieferantenkapazitäten**

$\sum_{i} Z_{lipt} \le bcap_{lpt}$

$∀ l,p \in VP, t$

In [78]:
for l in L:
  for p in VP:
    for t in T:
      scip.addCons(quicksum(Z[l,i,p,t] for i in I)<= bcap.get((l,p,t),0))

**(2) Produktionskapazitäten**

$\sum_{p \in FP} X_{ipt} \le pcap_{it} + O_{it}$

$∀ i, t$

In [79]:
for i in I:
  for t in T:
    scip.addCons(sum(X[i,p,t] for p in FP) <= pcap[i,t] + O[i,t])

**(3) Beschränkung der Zusatzkapazitäten**

$O_{it} \le pcap^+_{it}$

$∀ i, t$

In [80]:
for i in I:
  for t in T:
    scip.addCons(O[i,t] <= pcap_plus[i,t])

**(4) Lagerbilanzgleichung für die Fertigprodukte**

$B_{ipt} = B_{ip,t-1} + X_{ipt} - \sum_j Y_{ijpt}$

$∀ i,p \in FP, t$

In [81]:
T

[1, 2, 3, 4, 5]

In [82]:
for i in I:
  for p in FP:
    for t in T:
        scip.addCons(B[i,p,t] == B.get((i,p,t-1),0) + X[i,p,t] - sum(Y[i,j,p,t] for j in J))

**(5) Lagerbilanzgleichung für die Vorprodukte**

$B_{ipt} = B_{ip,t-1} + \sum_l Z_{lipt} - \sum_{p' \in A_{p} } a_{pp'} * X_{ip't}$

$∀ i,p \in VP, t$

In [83]:
for i in I:
  for p in VP:
    for t in T:
      scip.addCons(B[i,p,t] == B.get((i,p,t-1),0) + sum(Z[l,i,p,t] for l in L) - sum(a[p,pp] * X[i,pp,t] for pp in A[p]))

**(6) Nachfragebefriedigung**

$\sum_i Y_{ijpt} = d_{jpt}$

$∀ j,p \in FP, t$

In [84]:
for j in J:
  for t in T:
    for p in FP:
      scip.addCons(sum(Y[i,j,p,t] for i in I) == d[j,p,t])

## Berechnung Lösung

In [85]:
scip.optimize()
print(scip.getStatus())

optimal


In [86]:
scip.getObjVal()

30419.35

In [89]:
pcap

{('Curitiba', 1): 100,
 ('Quanzhou', 1): 250,
 ('East London', 1): 250,
 ('Curitiba', 2): 72,
 ('Quanzhou', 2): 253,
 ('East London', 2): 235,
 ('Curitiba', 3): 78,
 ('Quanzhou', 3): 168,
 ('East London', 3): 223,
 ('Curitiba', 4): 126,
 ('Quanzhou', 4): 278,
 ('East London', 4): 218,
 ('Curitiba', 5): 130,
 ('Quanzhou', 5): 150,
 ('East London', 5): 275}

In [87]:
for t in T:
  print(f"Woche: {t}")
  for i in I:
    for p in FP:
      val = round(scip.getVal(X[i,p,t]))
    if val > 0:
      print(f"Werk {i}")
      print(f"Produktion: {val}")
    print(f"Fertigwarenbestand: {round(scip.getVal(B[i,p,t]))}")
    print("\t")
  print("\n")


Woche: 1
Werk Curitiba
Produktion: 82
Fertigwarenbestand: 12
	
Werk Quanzhou
Produktion: 250
Fertigwarenbestand: 10
	
Werk East London
Produktion: 250
Fertigwarenbestand: 20
	


Woche: 2
Werk Curitiba
Produktion: 72
Fertigwarenbestand: 0
	
Werk Quanzhou
Produktion: 253
Fertigwarenbestand: 0
	
Werk East London
Produktion: 235
Fertigwarenbestand: 12
	


Woche: 3
Werk Curitiba
Produktion: 78
Fertigwarenbestand: 0
	
Werk Quanzhou
Produktion: 168
Fertigwarenbestand: 0
	
Werk East London
Produktion: 223
Fertigwarenbestand: 0
	


Woche: 4
Fertigwarenbestand: 0
	
Werk Quanzhou
Produktion: 216
Fertigwarenbestand: 26
	
Werk East London
Produktion: 218
Fertigwarenbestand: 0
	


Woche: 5
Werk Curitiba
Produktion: 119
Fertigwarenbestand: 0
	
Werk Quanzhou
Produktion: 150
Fertigwarenbestand: 0
	
Werk East London
Produktion: 275
Fertigwarenbestand: 0
	


